##### <span style="color:#483D8B; font-weight:bold;">Import Libraries</span>

!pip install tweet-preprocessor
!pip install pyspellchecker
!pip install symspellpy
!pip install transformers datasets
!pip install torch torchvision torchaudio --quiet
!pip install transformers datasets --quiet
!pip install -U accelerate
!pip install --upgrade pip
!pip install evaluate --quiet

!pip install hf_xet

!pip install --upgrade transformers --force-reinstall

!pip install numpy==1.26.4 --force-reinstall

!python -m pip install ipykernel
!python -m ipykernel install --user --name transformers452 --display-name "Python (Transformers 4.52)"

In [39]:
!pip install numpy==1.26.4 --force-reinstall

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.0
    Uninstalling numpy-2.3.0:
      Successfully uninstalled numpy-2.3.0


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.


In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\User\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\User\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\User\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\User\anaconda3\Lib\site-packages

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: initialization failed

In [ ]:
import

In [ ]:
import string
import re
import preprocessor as p
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from symspellpy.symspellpy import SymSpell, Verbosity
from sklearn.utils import resample

In [ ]:
from datasets import Dataset
from transformers import TrainingArguments
from transformers import Trainer
import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer

##### <span style="color:#483D8B; font-weight:bold;">Import Dataset</span>

In [ ]:
path = r"C:\Users\User\Desktop\Ironhack_DA\Final-project-hate-speech-detection-with-NLP\Dataset-Hate-Speech-Detection.csv"

In [ ]:
df = pd.read_csv(path, low_memory=False)

In [ ]:
df

Class explanation:

- 0 - Hate Speech
- 1 - Offensive Language
- 2 - Neither/Neutral content

In [ ]:
df.shape

In [ ]:
df.reset_index(drop=True, inplace=True)
df.index = df.index + 0

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

NOTE: no null values & consistent datatypes

##### <span style="color:#483D8B; font-weight:bold;">Cleaning tweet column</span>

In [ ]:
from IPython.display import display
pd.set_option('display.max_colwidth', None)

In [ ]:
display(df.iloc[:10, :2])

In [ ]:
df[df['class'] == 0]

In [ ]:
df.groupby('class').nunique()

In [ ]:
class_count = df['class'].value_counts()

ax = class_count.plot(kind='bar')

for i, value in enumerate(class_count):
    ax.text(i, value + 0.5, str(value), ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
display(df.iloc[:5, :2])

In [ ]:
nltk.download('stopwords')

df['tweet'].lower() #lower case
punc_to_remove = string.punctuations
df['tweet'].translate(str.maketrans('', '', PUNCT_TO_REMOVE))
" ".join(df['tweet'].split())
contractions.fix(df['tweet'])



In [ ]:
#df.tweet.str.lower(inplace = True)

#df['tweet'].replace(string.punctuation,'')

#def cleaned_tweet(text):
    # Remove punctuation
    #text = ''.join(char for char in text if char not in string.punctuation)
    #return text.lower()  # Optional: lowercasing

# Apply the cleaning function to the 'tweet' column
#df['tweet'] = df['tweet'].apply(cleaned_tweet)

In [ ]:
max_edit_distance_dictionary = 2
prefix_length = 7
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)

dictionary_path = "frequency_dictionary_en_82_765.txt"  # path to the downloaded dictionary
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.EMOJI)

stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

def correct_spelling_symspell(text):
    corrected_words = []
    for word in text.split():
        # Skip short words and stopwords to speed up
        if len(word) < 3 or word in stop_words:
            corrected_words.append(word)
            continue
        
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_words.append(suggestions[0].term)
        else:
            corrected_words.append(word)
    return " ".join(corrected_words)

def clean_tweet(text):
    text = p.clean(text)
    text = correct_spelling_symspell(text)
    pattern = r'([{}])\1+'.format(re.escape(punctuation))
    text = re.sub(pattern, r'\1', text)
    tokens = tokenizer.tokenize(text)
    words = [w for w in tokens if w not in stop_words]
    return " ".join(words)

df['tweet_cleaned'] = df['tweet'].apply(clean_tweet)

In [ ]:
df['tweet_cleaned'] = df['tweet_cleaned'].str.replace(r'!\s*rt\s*:', '', regex=True).str.replace(r"[^\w\s.,!?']", "", regex=True)

In [ ]:
df

##### <span style="color:#483D8B; font-weight:bold;">Model bulding</span>

##### <span style="color:#483D8B; font-weight:bold;">1 - Handling Imbalanced Data</span>

In [ ]:
df.groupby('class').nunique()

In [ ]:
df.drop('tweet', inplace = True, axis=1)

In [ ]:
df = df.drop_duplicates(subset='tweet_cleaned', keep='first').reset_index(drop=True)

In [ ]:
multi_class_tweets = df.groupby('tweet_cleaned')['class'].nunique()

tweets_with_multiple_classes = multi_class_tweets[multi_class_tweets > 1]

print(tweets_with_multiple_classes)

In [ ]:
df_0 = df[df['class'] == 0]   # All hate speech tweets
df_1 = df[df['class'] == 1]   # All offensive tweets
df_2 = df[df['class'] == 2]   # All neutral tweets

In [ ]:
max_size = df_1.shape[0]
max_size 

#there are duplicate rows

In [ ]:
df.groupby('class').nunique()

In [ ]:
df_0_upsampled = resample(df_0, replace=True, n_samples=max_size, random_state=42)
df_2_upsampled = resample(df_2, replace=True, n_samples=max_size, random_state=42)

In [ ]:
df_balanced = pd.concat([df_0_upsampled, df_2_upsampled, df_1])

In [ ]:
df_balanced.groupby('class').count()

In [ ]:
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_balanced.groupby('class').count()

##### <span style="color:#483D8B; font-weight:bold;">2 - RoBERTa model bulding</span>

Using a lighter version or Roberta

In [ ]:
df_balanced = df_balanced.rename(columns={"tweet_cleaned": "text", "class": "label"}) # a hugging face dataset

In [ ]:
dataset = Dataset.from_pandas(df_balanced)

In [ ]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_dataset

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=3)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,                    # Fast: just 1 epoch
    per_device_train_batch_size=8,        # Small batch = faster on CPU
    evaluation_strategy="epoch",          # Evaluate every epoch
    save_strategy="no",                   # Skip saving after each epoch
    logging_strategy="epoch",             # Show logs each epoch
    report_to="none",                     # No external logging (e.g., WandB)
)

In [ ]:
accuracy = evaluate.load("accuracy")

accuracy = evaluate.load("accuracy")  # Standard accuracy metric
f1 = evaluate.load("f1")              # F1 score — balances precision and recall

def compute_metrics(eval_pred):
    logits, labels = eval_pred             # Raw model outputs and true labels
    predictions = logits.argmax(axis=-1)  # Get predicted class by choosing max logit value
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels),
        "f1": f1.compute(predictions=predictions, references=labels, average="macro")
    }

In [ ]:
trainer = Trainer(
    model=model,                         # Our RoBERTa model
    args=training_args,                  # Training configuration
    train_dataset=tokenized_dataset["train"],  # Training data
    eval_dataset=tokenized_dataset["test"],    # Evaluation data
    tokenizer=tokenizer,                 # Tokenizer (helps with decoding & padding)
    compute_metrics=compute_metrics     # Function to calculate metrics during evaluation
)

In [ ]:
trainer.train()

##### <span style="color:#483D8B; font-weight:bold;">2 - BERT model bulding</span>